In [1]:
import pymysql
import pymssql
import os
import pandas as pd
import datetime

In [2]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
        )
sql_conn = ms_sql_con()

In [3]:
def calls_mysql_con_init():
    return pymysql.connect(
        host = '10.2.4.87',
        user = 'root',
        passwd = 'root',
        database = 'ml'
    )
calls_mysql_conn = calls_mysql_con_init()

In [23]:
def master_mysql_con_init():
    with open('mysql.pass','r') as file:
        mysql_pass = file.read().replace('\n', '')
        file.close()
    return pymysql.connect(
        host = '10.2.4.146', 
        user = 'asterisk', 
        passwd = mysql_pass,
        db = 'MICO_96'
    )
master_mysql_conn = master_mysql_con_init()

In [24]:
def call_mysql_con_init():
    with open('mysql.pass','r') as file:
        mysql_pass = file.read().replace('\n', '')
        file.close()
    return pymysql.connect(
        host = '10.2.4.146', 
        user = 'asterisk', 
        passwd = mysql_pass,
        db = 'asterisk'
    )
call_mysql_con = call_mysql_con_init()

In [40]:
def get_file_list(path):
    file_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_list.append(os.path.join(root, file))
            #file_list.append(root+'/'+file)
    return file_list

def get_file_date(file_list):
    file_date = []
    for file in file_list:
        file_date.append(datetime.datetime.fromtimestamp(os.path.getmtime(file)))
    return file_date

def get_file_name(file_list):
    path_name = []
    file_name = []
    for file in file_list:
        #file_name.append(file)
        file_name.append(os.path.basename(file))        
        path_name.append(os.path.dirname(file))
    return path_name, file_name

In [ ]:
def get_files(path):
    file_list = get_file_list(path)
    file_date = get_file_date(file_list)
    path, file_name = get_file_name(file_list)
    df = pd.DataFrame({'path':path, 'file_name':file_name, 'file_date':file_date})
    df.sort_values(by='file_date', inplace=True)
    return df

#### Queue

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
df = pd.read_sql(query, sql_conn)
#df

In [ ]:
df = pd.read_sql('select date, filename from queue where source_id=1', sql_conn)
df.head(3)

#### Transcribations

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
df = pd.read_sql(query, sql_conn)
#df

In [ ]:
query = 'select distinct linkedid, audio_file_name from transcribations '
query += "where audio_file_name='in_4999029955_2021-12-10-08-41-59rxtx.wav'"
df = pd.read_sql(query, sql_conn)
df

In [ ]:
query = 'select max(transcribation_date) from transcribations '
query += "where source_id=1"
df = pd.read_sql(query, sql_conn)
df

#### Calls

In [21]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='calls';"
#df = pd.read_sql(query, calls_mysql_conn)

,column_name


In [ ]:
### DELETE
#query = "delete from calls where call_date>'2021-12-10 00-00-00';"
#calls = pd.read_sql(query, calls_mysql_conn)

In [4]:
query = "SELECT distinct"
query += " call_date, ak, miko, mrm, incoming, linkedid, base_name"
query += " FROM calls"
query += " where call_date>'2021-12-10 10-10-00'"
query += " order by call_date;"
calls = pd.read_sql(query, calls_mysql_conn)
#calls = pd.read_sql("SELECT distinct * FROM calls where linkedid='1639126809.114061' order by call_date;", calls_mysql_conn)
#calls = pd.read_sql("SELECT * FROM calls where linkedid='1639033806.82183' order by call_date;", calls_mysql_conn)
calls

,call_date,ak,miko,mrm,incoming,linkedid,base_name
0,2021-12-10 10:10:02,0,1,0,1,1639120182.110147,1c_service
1,2021-12-10 10:10:21,0,1,0,0,1639120213.110184,1c_service
2,2021-12-10 10:10:21,0,1,0,0,1639120213.110184,1c_service_region
3,2021-12-10 10:10:21,0,1,0,0,1639120213.110184,1c_service_spb
4,2021-12-10 10:10:21,0,1,0,1,1639120213.110184,1c_service
...,...,...,...,...,...,...,...
1835,2021-12-10 12:25:49,0,1,0,0,1639128348.115182,1c_service
1836,2021-12-10 12:25:51,0,1,0,1,1639128348.115181,1c_service
1837,2021-12-10 12:25:57,1,0,1,0,1639128354.2367938,1c_service
1838,2021-12-10 12:26:17,1,0,1,0,1639128375.2367964,1c_service


#### Cdr call

In [28]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cdr';"
df = pd.read_sql(query, call_mysql_con)
#for idx, row in df.iterrows():
#    print(row.column_name)

In [ ]:
"select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"

#### PT1C_cdr_MICO master

In [38]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='PT1C_cdr_MICO';"
#query = "show tables;"
df = pd.read_sql(query, master_mysql_conn)
#df

In [36]:
query = "SELECT * from PT1C_cdr_MICO limit 1;"
df = pd.read_sql(query, master_mysql_conn)
df

,id,calldate,clid,src,dst,dcontext,lastapp,lastdata,duration,billsec,...,amaflags,accountcode,uniqueid,userfield,answer,end,recordingfile,peeraccount,linkedid,sequence
0,0,2019-02-10 18:30:06,"""4950000001"" <74991376253>",74991376253,s,Work_time_CC,Queue,"queue_oper,Ttr",163.848,163.847,...,DOCUMENTATION,None,1549812606.613008,,2019-02-10 18:30:06,2019-02-10 18:32:50,2019-02/10/in_74991376253_2019-02-10-18-30-07....,,1549812606.613008,709909


#### Files call

In [ ]:
files_path = 'call/saved_for_analysis/debug/call'
df = get_files(files_path)
df.to_csv('files_call.csv', index=False)
df.head(3)

#### Files master

In [ ]:
files_path = 'call/saved_for_analysis/debug/master'
df = get_files(files_path)
df.to_csv('files_call.csv', index=False)
df.head(3)